# IMAGE CRAWLER

In [149]:
import requests as req
from bs4 import *
from queue import Queue
import logging
import os
import uuid
from threading import Thread
import re
# from flask import abort, Flask, request, jsonify

In [150]:
#url = "https://www.airbnb.co.uk/s/Ljubljana--Slovenia/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Ljubljana%2C%20Slovenia&place_id=ChIJ0YaYlvUxZUcRIOw_ghz4AAQ&checkin=2020-11-01&checkout=2020-11-08&source=structured_search_input_header&search_type=autocomplete_click"
url2 = "http://4chan.org/"

In [160]:
logging.basicConfig(format='scraper - %(asctime)s - %(name)s - %(threadName)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)
# app = Flask(__name__)
tasks_results = {}


In [161]:
class Tasks:
    """Class for storing info for every task and return info and results"""

    def __init__(self, task_uuid, queue_urls):
        """
        Init new class copy
        :param task_uuid: UUID of task
        :param queue_urls: queue URLs
        """
        from time import time
        self.found_urls = {}
        self.urls_done = {}
        self.time_takes = None
        self.time_start = time()
        self.task_uuid = task_uuid
        for url in queue_urls:
            self.urls_done[url] = False
            self.found_urls[url] = []

    def prepare_url(self, src_url, url):
        """
        Formating URL
        :param src_url: root URL
        :param url: URL scraped
        :return: formated URL
        """
        import re
        if re.match(r'^[/]{2}.*', url):
            return f'http:{url}'
        elif re.match(r'^/\w+.*', url):
            return f'{src_url}{url}'
        elif not re.match(r'^http.*', url):
            return f'{src_url}/{url}'
        return url

    def _add_url(self, src_url, url):
        """
        Check if url fit info that needed, add it to results
        :param src_url: main URL
        :param url: URL scraped
        :return: None
        """
        url_needed = False
        for pat in ['.gif', '.jpg', '.png']:
            if pat in url:
                url_needed = True
                break
        if url_needed:
            if url not in self.found_urls[src_url]:
                self.found_urls[src_url].append(url)

    def add_res(self, src_url, urls):
        """
        Processing with scraped URL
        :param src_url: Main URL
        :param urls: URL scraped
        :return:
        """
        if isinstance(urls, str):
            self._add_url(src_url, urls)
        else:
            for url in urls:
                self._add_url(src_url, url)

    def finished(self, url):
        """
        Set URL as finished to scrape
        :param url: Main URL
        :return:
        """
        from time import time
        self.urls_done[url] = True
        self.time_takes = time() - self.time_start
        logging.info(f'{self.task_uuid} Took {self.time_takes}')

    def status(self):
        """
        Return JSON status
        :return:
        """
        return {"completed": sum(self.urls_done.values()),
                "inprogress": len(self.urls_done.values()) - sum(self.urls_done.values())}

    def __repr__(self):
        return str(self.status())

In [162]:
task = Tasks(task_id, url_list)
tasks_results[task_id] = task

NameError: name 'task_id' is not defined

In [124]:
def prepare_url(src_url, url):
    import re
    if re.match(r'^[/]{2}.*', url):
        return f'http:{url}'
    elif re.match(r'^/\w+.*', url):
        return f'{src_url}{url}'
    elif not re.match(r'^http.*', url):
        return f'{src_url}/{url}'
    return url

In [137]:
found_urls = {}


In [142]:
def _add_url(src_url, url):
    url_needed = False
    for pat in ['.gif', '.jpg', '.png']:
        if pat in url:
            url_needed = True
            break
    if url_needed:
        if url not in found_urls[src_url]:
            found_urls[src_url].append(url)

In [143]:
def add_res(src_url, urls):
    if isinstance(urls, str):
        _add_url(src_url, urls)
    else:
        for url in urls:
            _add_url(src_url, url)

In [140]:
## function to get img links
def parse_url(src_url, url, deep_lv = 1):
    ## req will send the request to this link and get the data in text format from this link webpage
    request = req.get(url)
    ## this will change text into html format
    soup = BeautifulSoup(request.text, "html.parser")
    hrefs = soup.find_all(href=True)
    ## grab all img links and append into links list
    images = soup.find_all('img')
    add_res(src_url, [prepare_url(url, link["src"]) for link in images])
    
    if deep_lv > 0:
        for href in hrefs:
            parse_url(src_url, prepare_url(url, href["href"]), deep_lv=deep_lv - 1)
        
        
    return links
    

In [141]:
links = parse_url(url2, url2)
links

KeyError: 'http://4chan.org/'

In [121]:
href_links = {}
src_url = url2
for href in hrefs:
    href_links[src_url] = parse_url(src_url, prepare_url(url, href["href"]) )
     
href_links

{'http://4chan.org/': (['https://a0.muscache.com/im/pictures/c12d27de-d2a0-485c-b70a-a65648f7b596.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/d3553d7c-97a2-487e-907f-a238eaaaba39.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/59738627-a049-4cec-9e21-2f31302c011c.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/f5d481b3-5451-4e7e-bf70-9efef4ace583.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/68ef0fea-4760-45bd-9fa0-1ee390e10698.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/b53d6945-24a8-4ebd-bd37-9f16f0720c75.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/ed76f499-b27c-4f82-bb96-7ff540534a22.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/5eb4eba3-d4a5-42a9-8f76-c2c6f733c359.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/2f799111-6fc1-4c98-a3af-b75ae16a9ab9.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/bc3d3c23-f226-47fc-94fa-ad92ae9be7c9.jpg?im_w=720',
   'https://a0.muscache.com/im/pictures/miso/Hosting-300032

In [122]:
for href in hrefs:
    a, b = parse_url(url2, prepare_url(src_url, href["href"]))
    print(a)

[]
[]
[]
[]
[]
[]
['http://s.4cdn.org/image/fp/logo-transparent.png', 'http://i.4cdn.org/vg/1619143988597s.jpg', 'http://i.4cdn.org/co/1619794920026s.jpg', 'http://i.4cdn.org/int/1619801208665s.jpg', 'http://i.4cdn.org/a/1619797903675s.jpg', 'http://i.4cdn.org/his/1619798056911s.jpg', 'http://i.4cdn.org/tg/1619728629944s.jpg', 'http://i.4cdn.org/mu/1619777499006s.jpg', 'http://i.4cdn.org/mlp/1618945111323s.jpg']
['http://s.4cdn.org/image/fp/logo-transparent.png', 'http://i.4cdn.org/co/1619612468273s.jpg', 'http://i.4cdn.org/tg/1619762396533s.jpg', 'http://i.4cdn.org/vg/1619759863642s.jpg', 'http://i.4cdn.org/vt/1619748245598s.jpg', 'http://i.4cdn.org/int/1619801816528s.jpg', 'http://i.4cdn.org/v/1619750964327s.jpg', 'http://i.4cdn.org/toy/1619758395696s.jpg', 'http://i.4cdn.org/g/1619791909957s.jpg']
['http://s.4cdn.org/image/contest_banners/696037243d9540e1d80c4ab98472afcafe0a7906.png', 'http://i.4cdn.org/a/1619796159406s.jpg', 'http://i.4cdn.org/a/1619805247369s.jpg', 'http://i.4cdn.

In [45]:
parse_url('http://boards.4channel.org/a/')

(['http://s.4cdn.org/image/contest_banners/0c3e75f8b0f48dc15a11a8ae95cd9a72d0472ce9.jpg',
  'http://i.4cdn.org/a/1619562687502s.jpg',
  'http://i.4cdn.org/a/1619639613089s.jpg',
  'http://i.4cdn.org/a/1619639855222s.jpg',
  'http://i.4cdn.org/a/1619639880474s.jpg',
  'http://i.4cdn.org/a/1619634231286s.jpg',
  'http://i.4cdn.org/a/1619629740764s.jpg',
  'http://i.4cdn.org/a/1619639312653s.jpg',
  'http://i.4cdn.org/a/1619621649255s.jpg',
  'http://i.4cdn.org/a/1619639859167s.jpg',
  'http://i.4cdn.org/a/1619639900336s.jpg',
  'http://i.4cdn.org/a/1619639924854s.jpg',
  'http://i.4cdn.org/a/1619638017705s.jpg',
  'http://i.4cdn.org/a/1619639865936s.jpg',
  'http://i.4cdn.org/a/1619639870232s.jpg',
  'http://i.4cdn.org/a/1619639915080s.jpg',
  'http://i.4cdn.org/a/1619607200438s.jpg',
  'http://i.4cdn.org/a/1619639656741s.jpg',
  'http://i.4cdn.org/a/1619639793189s.jpg',
  'http://i.4cdn.org/a/1619639919349s.jpg',
  'http://i.4cdn.org/a/1619625835772s.jpg',
  'http://i.4cdn.org/a/1619639

In [ ]:
## function to get img links
def parse_url(src_url, url, deep_lv=1):
    ## req will send the request to this link and get the data in text format from this link webpage
    request = req.get(url)
    ## this will change text into html format
    soup = BeautifulSoup(request.text, "html.parser")
    hrefs = soup.find_all(href=True)
    ## grab all img links and append into links list
    images = soup.find_all('img')
    links = [prepare_url(url, link["src"]) for link in images]
    
    if deep_lv > 0:
        for href in hrefs:
            parse_url(src_url, prepare_url(url, href["href"]), deep_lv = deep_lv-1)
        
    return links
    